In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import tensorboard
import os
import matplotlib as plt

# Defining Simple CNN (with LeakyReLU)

In [73]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 300, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(300, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.leaky_relu(self.conv1(x)))
        x = self.pool(F.leaky_relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [74]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = Net()
net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9)

# Downloading  and  preparing  CIFAR-10  dataset 

In [70]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

#images, labels = next(iter(trainloader))

Files already downloaded and verified
Files already downloaded and verified


# Training the network (with SGD as loss function)

In [75]:
writer = SummaryWriter()
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    total = 0
    correct = 0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)
        #inputs, labels = data
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs).to(device)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # print statistics
        running_loss += loss.item()
        writer.add_scalar('Loss/train', loss.item(), i)
        writer.add_scalar('Accuracy/train', correct/total, i)
       
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
            print("Accuracy: ", correct/total)

print('Finished Training')

grid = torchvision.utils.make_grid(inputs)
writer.add_image('images', grid, 0)
writer.add_graph(net, inputs)
writer.close()

[1,  2000] loss: 2.291
Accuracy:  0.139125
[1,  4000] loss: 2.176
Accuracy:  0.178875
[1,  6000] loss: 1.979
Accuracy:  0.21391666666666667
[1,  8000] loss: 1.878
Accuracy:  0.2401875
[1, 10000] loss: 1.779
Accuracy:  0.263475
[1, 12000] loss: 1.712
Accuracy:  0.28191666666666665
[2,  2000] loss: 1.601
Accuracy:  0.410875
[2,  4000] loss: 1.562
Accuracy:  0.4165625
[2,  6000] loss: 1.549
Accuracy:  0.42383333333333334
[2,  8000] loss: 1.512
Accuracy:  0.42934375
[2, 10000] loss: 1.458
Accuracy:  0.438825
[2, 12000] loss: 1.462
Accuracy:  0.4433958333333333
[3,  2000] loss: 1.431
Accuracy:  0.478125
[3,  4000] loss: 1.387
Accuracy:  0.487875
[3,  6000] loss: 1.377
Accuracy:  0.495375
[3,  8000] loss: 1.367
Accuracy:  0.49671875
[3, 10000] loss: 1.362
Accuracy:  0.49845
[3, 12000] loss: 1.318
Accuracy:  0.5023541666666667
[4,  2000] loss: 1.301
Accuracy:  0.52775
[4,  4000] loss: 1.280
Accuracy:  0.5354375
[4,  6000] loss: 1.282
Accuracy:  0.53725
[4,  8000] loss: 1.260
Accuracy:  0.5397

# Testing the network on testset and reporting accuracy (with SGD)

In [78]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        #images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 67 %


# Changing optimizer to Adam and running expirement again

In [81]:
optimizer = optim.Adam(net.parameters(), lr=0.0001)

In [82]:
writer = SummaryWriter()
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    total = 0
    correct = 0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)
        #inputs, labels = data
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs).to(device)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # print statistics
        running_loss += loss.item()
        writer.add_scalar('Loss/train', loss.item(), i)
        writer.add_scalar('Accuracy/train', correct/total, i)
       
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
            print("Accuracy: ", correct/total)

print('Finished Training')

grid = torchvision.utils.make_grid(inputs)
writer.add_image('images', grid, 0)
writer.add_graph(net, inputs)
writer.close()

[1,  2000] loss: 0.945
Accuracy:  0.66175
[1,  4000] loss: 0.969
Accuracy:  0.661625
[1,  6000] loss: 0.957
Accuracy:  0.6609166666666667
[1,  8000] loss: 0.937
Accuracy:  0.664
[1, 10000] loss: 0.947
Accuracy:  0.6646
[1, 12000] loss: 0.921
Accuracy:  0.6661666666666667
[2,  2000] loss: 0.871
Accuracy:  0.69625
[2,  4000] loss: 0.882
Accuracy:  0.693625
[2,  6000] loss: 0.866
Accuracy:  0.6925833333333333
[2,  8000] loss: 0.860
Accuracy:  0.6946875
[2, 10000] loss: 0.872
Accuracy:  0.693825
[2, 12000] loss: 0.894
Accuracy:  0.6921875
[3,  2000] loss: 0.791
Accuracy:  0.72475
[3,  4000] loss: 0.830
Accuracy:  0.716875
[3,  6000] loss: 0.806
Accuracy:  0.7146666666666667
[3,  8000] loss: 0.808
Accuracy:  0.71428125
[3, 10000] loss: 0.810
Accuracy:  0.71525
[3, 12000] loss: 0.802
Accuracy:  0.7152291666666667
[4,  2000] loss: 0.724
Accuracy:  0.749875
[4,  4000] loss: 0.749
Accuracy:  0.7450625
[4,  6000] loss: 0.767
Accuracy:  0.74125
[4,  8000] loss: 0.755
Accuracy:  0.7391875
[4, 1000

# Testing the network on testset and reporting accuracy (with adam)

In [83]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        #images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 72 %


AttributeError: module 'matplotlib' has no attribute 'show'